In [33]:
NEO4J_URI="neo4j+s://2f39d47e.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="_YGbxuqFNJevkNthWRnhpGCxsSfwgTy_k65IxKmNZu0"

In [34]:
import os
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [35]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD)
graph

In [36]:
moview_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))


"""
graph.query(moview_query)

[]

In [37]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000023D381DAAA0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023D381D96C0>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [38]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(graph=graph,llm=llm,exclude_types=["Genre"],verbose=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x0000023D3815C5E0>, cypher_generation_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'schema'], template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000023D381DAAA0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023D381D96C0>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [39]:
chain.graph_schema

'Node properties are the following:\nMovie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT},Person {name: STRING}\nRelationship properties are the following:\n\nThe relationships are the following:\n(:Person)-[:DIRECTED]->(:Movie),(:Person)-[:ACTED_IN]->(:Movie)'

In [42]:
from langchain_core.prompts import PromptTemplate

# Define the prompt template
prompt_template = """
You are a Neo4j expert. Given an input question and the following schema: {schema}, create a syntactically very accurate Cypher query.

Examples:
{examples}

User input: {question}
Cypher query:
"""

# Define examples in a format that the prompt template will understand
examples = """
1. Question: How many artists are there?
   Cypher query: MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)

2. Question: Which actors played in the movie Casino?
   Cypher query: MATCH (m:Movie {title: 'Casino'})<-[:ACTED_IN]-(a) RETURN a.name

3. Question: How many movies has Tom Hanks acted in?
   Cypher query: MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)

4. Question: List all the genres of the movie Schindler's List
   Cypher query: MATCH (m:Movie {title: 'Schindler\\'s List'})-[:IN_GENRE]->(g:Genre) RETURN g.name

5. Question: Which actors have worked in movies from both the comedy and action genres?
   Cypher query: MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name
"""

# Instantiate the PromptTemplate
template = PromptTemplate(
    template=prompt_template,
    input_variables=["question", "schema", "examples"]
)

# Format the prompt
formatted_prompt = template.format(
    question="How many artists are there?",
    schema="foo",
    examples=examples
)

print(formatted_prompt)



You are a Neo4j expert. Given an input question and the following schema: foo, create a syntactically very accurate Cypher query.

Examples:

1. Question: How many artists are there?
   Cypher query: MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)

2. Question: Which actors played in the movie Casino?
   Cypher query: MATCH (m:Movie {title: 'Casino'})<-[:ACTED_IN]-(a) RETURN a.name

3. Question: How many movies has Tom Hanks acted in?
   Cypher query: MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)

4. Question: List all the genres of the movie Schindler's List
   Cypher query: MATCH (m:Movie {title: 'Schindler\'s List'})-[:IN_GENRE]->(g:Genre) RETURN g.name

5. Question: Which actors have worked in movies from both the comedy and action genres?
   Cypher query: MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.na

In [ ]:
response = chain.invoke({"query": "How many artists are there?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person) RETURN count(DISTINCT p)
Full Context:
[{'count(DISTINCT p)': 1239}]

> Finished chain.


{'query': 'How many artists are there?',
 'result': 'There are 1239 artists. \n'}

In [ ]:
prompt

FewShotPromptTemplate(input_variables=['question'], examples=[{'question': 'How many artists are there?', 'query': 'MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)'}, {'question': 'Which actors played in the movie Casino?', 'query': "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name"}, {'question': 'How many movies has Tom Hanks acted in?', 'query': "MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)"}, {'question': "List all the genres of the movie Schindler's List", 'query': "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name"}, {'question': 'Which actors have worked in movies from both the comedy and action genres?', 'query': "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name"}], example_prompt=PromptTemplate(input_variables=['query', 'question'], template='Us

In [ ]:
llm
chain=GraphCypherQAChain.from_llm(graph=graph,llm=llm,cypher_prompt=prompt,verbose=True)

In [ ]:
chain.invoke("Which actors have worked in movies from both the comedy and action genres?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (a:Person)-[:ACTED_IN]->(m:Movie)-[:IN_GENRE]->(g1:Genre {name: 'Comedy'}),
     (a)-[:ACTED_IN]->(m2:Movie)-[:IN_GENRE]->(g2:Genre {name: 'Action'})
RETURN DISTINCT a.name

Full Context:
[{'a.name': 'Angela Bassett'}, {'a.name': 'Julia Ormond'}, {'a.name': 'Harrison Ford'}, {'a.name': 'Jim Carrey'}, {'a.name': 'Wesley Snipes'}, {'a.name': 'Woody Harrelson'}, {'a.name': 'Gene Hackman'}, {'a.name': 'Rene Russo'}, {'a.name': 'John Travolta'}, {'a.name': 'Nicole Kidman'}]

> Finished chain.


{'query': 'Which actors have worked in movies from both the comedy and action genres?',
 'result': "I don't know the answer. \n"}